In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

T_f = 350.0

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSR_avg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFNSR_avg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=T_f)
abTempF2S = InterfaceData(size=inodes, value=T_f)
FluxS = InterfaceData(size=inodes, value=0.0)
FluxF = InterfaceData(size=inodes, value=0.0)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-5)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FNSR_avg%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FNSR_avg%i.log' % k, mode='a')

abtemp_prev = abTempF.curr.copy()
fluxs_prev = FluxS.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    abtemp_prev[:] = abTempF.curr
    fluxs_prev[:] = FluxS.curr
    
    FluxF.curr[:] = FluxS.curr[s2f]
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=FluxF.curr
    )
    
    while True:
        # back up previous interface value
        FluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = 0.5*(abTempF.curr[f2s]+abtemp_prev[f2s])
        # update solid robin interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        FluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        FluxF.curr[:] = FluxS.curr[s2f]
        
        # update residual
        FluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(FluxF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(FluxF)
                under_relax.update_solution(FluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=0.5*(FluxF.curr+fluxs_prev[s2f])
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=11.
step=1, pc_iterations=13.
step=2, pc_iterations=13.
step=3, pc_iterations=13.
step=4, pc_iterations=13.
step=5, pc_iterations=14.
step=6, pc_iterations=13.
step=7, pc_iterations=14.
step=8, pc_iterations=13.
step=9, pc_iterations=14.
step=10, pc_iterations=13.
step=11, pc_iterations=14.
step=12, pc_iterations=13.
step=13, pc_iterations=14.
step=14, pc_iterations=13.
step=15, pc_iterations=14.
step=16, pc_iterations=13.
step=17, pc_iterations=14.
step=18, pc_iterations=13.
step=19, pc_iterations=14.
step=20, pc_iterations=13.
step=21, pc_iterations=14.
step=22, pc_iterations=13.
step=23, pc_iterations=14.
step=24, pc_iterations=13.
step=25, pc_iterations=14.
step=26, pc_iterations=13.
step=27, pc_iterations=14.
step=28, pc_iterations=13.
step=29, pc_iterations=14.
step=30, pc_iterations=13.
step=31, pc_iterations=14.
step=32, pc_iterations=13.
step=33, pc_iterations=14.
step=34, pc_iterations=13.
step=35, pc_iterations=14.
step=36, pc_iterations=13.
step=37, pc

step=297, pc_iterations=13.
step=298, pc_iterations=13.
step=299, pc_iterations=13.
step=300, pc_iterations=13.
step=301, pc_iterations=13.
step=302, pc_iterations=13.
step=303, pc_iterations=13.
step=304, pc_iterations=13.
step=305, pc_iterations=13.
step=306, pc_iterations=13.
step=307, pc_iterations=13.
step=308, pc_iterations=13.
step=309, pc_iterations=13.
step=310, pc_iterations=13.
step=311, pc_iterations=13.
step=312, pc_iterations=13.
step=313, pc_iterations=13.
step=314, pc_iterations=13.
step=315, pc_iterations=13.
step=316, pc_iterations=13.
step=317, pc_iterations=13.
step=318, pc_iterations=13.
step=319, pc_iterations=13.
step=320, pc_iterations=13.
step=321, pc_iterations=13.
step=322, pc_iterations=13.
step=323, pc_iterations=13.
step=324, pc_iterations=13.
step=325, pc_iterations=13.
step=326, pc_iterations=13.
step=327, pc_iterations=13.
step=328, pc_iterations=13.
step=329, pc_iterations=13.
step=330, pc_iterations=13.
step=331, pc_iterations=13.
step=332, pc_iterati

step=590, pc_iterations=13.
step=591, pc_iterations=13.
step=592, pc_iterations=13.
step=593, pc_iterations=13.
step=594, pc_iterations=13.
step=595, pc_iterations=13.
step=596, pc_iterations=13.
step=597, pc_iterations=13.
step=598, pc_iterations=13.
step=599, pc_iterations=13.
step=600, pc_iterations=13.
step=601, pc_iterations=13.
step=602, pc_iterations=13.
step=603, pc_iterations=13.
step=604, pc_iterations=13.
step=605, pc_iterations=13.
step=606, pc_iterations=13.
step=607, pc_iterations=13.
step=608, pc_iterations=13.
step=609, pc_iterations=13.
step=610, pc_iterations=13.
step=611, pc_iterations=13.
step=612, pc_iterations=13.
step=613, pc_iterations=13.
step=614, pc_iterations=13.
step=615, pc_iterations=13.
step=616, pc_iterations=13.
step=617, pc_iterations=13.
step=618, pc_iterations=13.
step=619, pc_iterations=13.
step=620, pc_iterations=13.
step=621, pc_iterations=13.
step=622, pc_iterations=13.
step=623, pc_iterations=13.
step=624, pc_iterations=13.
step=625, pc_iterati

step=883, pc_iterations=13.
step=884, pc_iterations=13.
step=885, pc_iterations=13.
step=886, pc_iterations=13.
step=887, pc_iterations=13.
step=888, pc_iterations=13.
step=889, pc_iterations=13.
step=890, pc_iterations=13.
step=891, pc_iterations=13.
step=892, pc_iterations=13.
step=893, pc_iterations=13.
step=894, pc_iterations=13.
step=895, pc_iterations=13.
step=896, pc_iterations=13.
step=897, pc_iterations=13.
step=898, pc_iterations=13.
step=899, pc_iterations=13.
step=900, pc_iterations=13.
step=901, pc_iterations=13.
step=902, pc_iterations=13.
step=903, pc_iterations=13.
step=904, pc_iterations=13.
step=905, pc_iterations=13.
step=906, pc_iterations=13.
step=907, pc_iterations=13.
step=908, pc_iterations=13.
step=909, pc_iterations=13.
step=910, pc_iterations=13.
step=911, pc_iterations=13.
step=912, pc_iterations=13.
step=913, pc_iterations=13.
step=914, pc_iterations=13.
step=915, pc_iterations=13.
step=916, pc_iterations=13.
step=917, pc_iterations=13.
step=918, pc_iterati

step=1170, pc_iterations=13.
step=1171, pc_iterations=13.
step=1172, pc_iterations=13.
step=1173, pc_iterations=13.
step=1174, pc_iterations=13.
step=1175, pc_iterations=13.
step=1176, pc_iterations=13.
step=1177, pc_iterations=13.
step=1178, pc_iterations=13.
step=1179, pc_iterations=13.
step=1180, pc_iterations=13.
step=1181, pc_iterations=13.
step=1182, pc_iterations=13.
step=1183, pc_iterations=13.
step=1184, pc_iterations=13.
step=1185, pc_iterations=13.
step=1186, pc_iterations=13.
step=1187, pc_iterations=13.
step=1188, pc_iterations=13.
step=1189, pc_iterations=13.
step=1190, pc_iterations=13.
step=1191, pc_iterations=13.
step=1192, pc_iterations=13.
step=1193, pc_iterations=13.
step=1194, pc_iterations=13.
step=1195, pc_iterations=13.
step=1196, pc_iterations=13.
step=1197, pc_iterations=13.
step=1198, pc_iterations=13.
step=1199, pc_iterations=13.
step=1200, pc_iterations=13.


In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
print(theta)
sio.savemat(
    'numeric_res/kFNSR_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.11249305 0.07142851 0.05910728 0.05968218 0.05850541 0.05614825
 0.0543537  0.05266204 0.05046612 0.04921747 0.0476877  0.0464839
 0.04494552 0.04378868 0.04248578 0.0414995  0.04027275 0.03924578
 0.03832948 0.03726314 0.03641074 0.03547129 0.03461478 0.03384535
 0.03299634 0.03222534 0.03142999 0.03075508 0.03013251 0.02946392
 0.02880564 0.02821273 0.0276193  0.02700366 0.02646285 0.02596115
 0.02540532 0.02489842 0.02437587 0.02392119 0.02349126 0.023068
 0.02263077 0.02221246 0.02180514 0.02142647 0.02102435 0.0206847
 0.02034313 0.01996704 0.01962505 0.01931821 0.01900731 0.018681
 0.01840836 0.01811166 0.01785019 0.01757873 0.01730001 0.01706917
 0.01683759 0.0165788  0.01634562 0.0161351  0.0159408  0.01570702
 0.01553385 0.01534029 0.01517327 0.01500375 0.01483111 0.01468916
 0.0145517  0.0143905  0.0142581  0.01414173 0.01404349 0.01391642
 0.01384335 0.01375132 0.01369104 0.01363004 0.01358007 0.01352251
 0.01350272 0.01349338 0.01347922 0.01350597 0.01354106 0.0135829
 0